In [1]:
#default_exp constraints

# Battery Constraints

The solution must meet constraints of the battery:

- Must only charge between 00:00-15:30
- Must only discharge between 15:30--21:00
- Must not charge or discharge between 21:00--00:00
- Battery must be empty at 00:00 each day

<br>

### Imports

In [2]:
#exports
import numpy as np
import pandas as pd

### Converting a charging schedule to capacity

The solution is given in terms of the battery charge/discharge schedule, but it is also necessary to satisfy constraints on the capacity of the battery (see below). 

The charge is determined by $C_{t+1} = C_{t} + 0.5B_{t}$

Note that we generally the initial value for capacity is 0 (the battery starts off empty). Change `init_value` if not.

In [3]:
#exports

def charge_to_capacity(charge_schedule, init_value=0):
    capacity = np.append(np.array([init_value]), np.cumsum(charge_schedule[:-1]/2))
    return capacity

A simple example: 

In [4]:
b = np.array([2.5, 0.5, 0, -1, 0])
charge_to_capacity(b)

array([0.  , 1.25, 1.5 , 1.5 , 1.  ])

### Determine whether a solution meets constraints

The battery schedule must meet constraints such that all at timesteps the charging rate $B$ and capacity $C$ satisfy:

* $B_{min} \leq B \leq B_{max}$ 
* $0 \leq C \leq C_{max}$
 
In addition, we can only charge the battery between periods 1 (00:00) and 31 (15:00) inclusive, and discharge between periods 32 (15:30) and 42 (20:30) inclusive. For periods 43 to 48, there should be no activity, and the day must start with $C=0$. 


In [5]:
#exports

def schedule_is_legal(schedule, B_min=-2.5, B_max=2.5, C_min=0, C_max=6, 
                      charge_times=('00:00', '15:00'), discharge_times=('15:30', '20:30'),
                      no_activity_times=('21:00', '23:30'), tolerance=1e-09):
    """
    Determine if a battery schedule meets constraints
    """
        
    charge = schedule.charge_MW.values
    capacity = charge_to_capacity(schedule.charge_MW.values)
    schedule['capacity'] = capacity
    schedule = schedule.set_index(pd.to_datetime(schedule.index))
    
    if not np.all((charge >= B_min-tolerance) & (charge <= (B_max+tolerance))): # charge constraints
        print("Charge constraint violated")
        return False
    elif not np.all((capacity >= C_min-tolerance) & (capacity <= (C_max+tolerance))): # capacity constraints
        print("Capacity constraint violated")
        return False
    elif np.any(schedule.between_time(discharge_times[0], discharge_times[1]).charge_MW.values > (0+tolerance)): # Discharge between discharge_times
        print("Attempted to charge during the discharge period")
        return False
    elif np.any(schedule.between_time(charge_times[0], charge_times[1]).charge_MW.values < (0-tolerance)): # Charge between charge_times 
        print("Attempted to discharge during the charge period")
        return False
    elif np.logical_not(
                        np.any(
                               np.isclose(schedule.between_time(no_activity_times[0], no_activity_times[1]).charge_MW.values, 0))): # No activity between no_activity_times 
        print("Attempted to discharge or charge during the no activity period")
        return False
    elif np.logical_not(np.any(np.isclose(schedule.between_time('00:00', '00:00').capacity.values, 0))): # Must be empty at 00:00
        print("Battery is not empty at 00:00")
        return False
    else:
        return True
    

Testing out on a random schedule 

In [6]:
example_schedule = pd.read_csv('../data/raw/example_valid_schedule.csv')

print(f"Is schedule legal? {schedule_is_legal(example_schedule)}")

Attempted to discharge during the charge period
Is schedule legal? False


<br>

Finally we'll export the relevant code to our `batopt` module

In [7]:
#hide
from nbdev.export import notebook2script
    
notebook2script()

Converted 00-utilities.ipynb.
Converted 01-data-cleaning.ipynb.
Converted 02-battery-discharge.ipynb.
Converted 03-battery-charge.ipynb.
Converted 04-battery-constraints.ipynb.
Converted 05-evaluation.ipynb.
Converted 06-pipeline.ipynb.
